# Event Flood Inundation Mapping

This notebook uses the Wildcat Creek (near Manhattan, KS) Labor Day flood event in 2018 to demonstarte how to map the flood event using FLDPLN tiled library.

## Import Modules and Packages

In [1]:
import sys
import time
# import DASK libraries
from dask.distributed import Client, LocalCluster
from dask import visualize

# import the mapping and gauge modules from the fldpln package
from fldpln.mapping import *
from fldpln.gauge import *

## Setup Input Tiled Library and Output Folders

Here we setup the folder under which tiled libraries (organized as folders) are located. We also setup the output folder (i.e., outputFolder) under which a map folder and a 'scratch' folder are created. The map folder, which is specified later, contains all inundation depth maps. The scratch folder stores temporary files.

In [ ]:
# tiled library folder
libFolder =  'E:/fldpln/sites/wildcat_10m_3dep/tiled_snz_library' # wildcat
# libFolder =  'E:/fldpln/sites/verdigris_10m/tiled_snz_library'

# libraries to be mapped
allLibNames = ['seglib_py'] # wildcat
# allLibNames = ['lib_fldsensing'] # verdigris

# Set output folder
outputFolder = 'E:/fldpln/sites/wildcat_10m_3dep/maps'
# outputFolder = 'E:/fldpln/sites/verdigris_10m/maps'

## Prepare Gauge Stage and Calculate Gauge Depth of Flow (DOF)

Here we obtain and prepare flood event stages from stream gauges. The stage at a gauge typically refers to the gauge's datum, which is not necessary of the stream bed elevation which is based on a certain vertical datum. In order to use gauge stage in a FLDPLN library, we need to make sure that gauge stage elevation (gauge + stage) and FSP's filled elevation are based on the same vertical datum. The depth of flow (DOF) at the FSP can then be calculated as the difference. The Wildcat Creek DEM and FLDPLN library are based on the NAVD88 vertical datum. So gauge stage elevations need to be based on the vertical datum too to calculate the DOFs at those gauges. 

### Gauge Stage from AHPS and USGS

Both USGS and NWS AHPS maintain stream gauges which record past flood stages. There are three AHPS and USGS gauges ([WKCK1](https://water.weather.gov/ahps2/hydrograph.php?wfo=top&gage=wkck1), [MWCK1](https://water.weather.gov/ahps2/hydrograph.php?wfo=top&gage=MWCK1), [MSTK1](https://water.weather.gov/ahps2/hydrograph.php?wfo=top&gage=MSTK1)) on the Wildcat Creek that record the 2018 Labor Day flood event. Here we use the maximum Labor Day flood event stages at those gauges to map the maximum inundation extent and depth of the event.

#### Event Stage from AHPS Historic Crests

The flood stage for the 2018 Labor Day flood event in 2018 are available as AHPS historic crests at those gauges [WKCK1](https://water.noaa.gov/gauges/WKCK1), [MWCK1](https://water.weather.gov/ahps2/hydrograph.php?wfo=top&gage=MWCK1) and [MSTK1](https://water.weather.gov/ahps2/hydrograph.php?wfo=top&gage=MSTK1). Excel file wildcat_gauges_albers_meters.xlsx has several sheets which store both gauge information (for example, gauge datum) and the event statges with different gauge combinations. The key fields needed for those gauges are: stationid, x, y, and stage_elevation

Note that most USGS and AHPS gauge stages are measured in feet and **Make sure that gauge coordinates are in the same coordinate system of the library and gauge stages are also in the same vertical unit of the library.** 

In [ ]:
# Wildcat Creek gauges
# gaugeStageFileName = 'wildcat_gauges.xlsx' # KS LiDAR DEM in UTM with vertical unit in feet
gaugeStageFileName = 'wildcat_gauges_albers_meters.xlsx' # 3DEP DEM in Albers with vertical unit in meters
sheetName = 'ThreeGauges' # all 3 gauges
# sheetName = 'TwoDsGauges' # 2 downstream gauges
# sheetName = 'MSTK1' # the last downstream gauge used in HEC-RAS model

# # Verdigris gauges
# gaugeStageFileName = 'VerdigrisGauges.xlsx' # CFVK1 is not a USGS gauge and its stage is from AHPS historic crests
# sheetName = '2019Flood' # gauges used for 2019 flood 

# read gauge file
gaugeStages = pd.read_excel(gaugeStageFileName, sheet_name=sheetName) 
# print(gaugeStages)

# Need to calculate gauge stage elevation if necessary!

# keep only necessary fields from gauges
keptFields = ['stationid','x','y','stage_elevation']
gaugeWithStageElevations = gaugeStages[keptFields]
print(gaugeWithStageElevations)

#### Event Stage from USGS NWIS

We can also get event maximum stage directly from USGS NWIS to check the historic crests from AHPS. Note that the stages are in feet and we need to convert stages to stage elevation before using it in flood inundation mapping.

In [ ]:
# Wildcat Creek 3 USGS gauges (in the order from upstream to downstream)
usgsIds = ['06879805','06879810','06879815'] 
ahpsIds = ['WKCK1','MWCK1','MSTK1']

# A period between two dates: Wildcat Creek Sep.3 2018 flood event
instStages = GetUsgsGaugeStageFromWebService(usgsIds,startDate='2018-09-02',endDate='2018-09-04')
print(instStages)

# find the max stage within the time period
maxStages = instStages.groupby(['stationid'],as_index=False).agg({'stage_ft':'max'})
# find the most recent time with the max stage
tdf = pd.merge(instStages, maxStages, how='inner', on=['stationid','stage_ft'])
gaugeStagesFromNwis = tdf.groupby(['stationid'], as_index=False).agg({'stationid':'first','stage_ft':'first','stage_time':'max'})
print(gaugeStagesFromNwis)

### Synthetic Gauge Stage from the National Water Model and HAND

HAND FIM uses NWM's discharge and turn it into stage. Here we use HAND reach stage to run FLDPLN for the event. Concepually, we turn reach stage into a synthetic gauge located at the either the mid-point or the outlet of the reach. Selecting the HAND reaches and sythteric gauge location is done by graduate student David Weiss manually for the Wildcat Creek example. Those sytheteic gauges can be treated as USGS/AHPS gauges. The key fields needed are: stationid, x, y, and stage_elevation.  Note that we assume the HAND reach stage elevation is the same as the FLDPLN library DEM. 

In [ ]:
# Synthetic FSP gauges from NWC reach stage
# gaugeStageFileName = 'wildcat_gauges.xlsx'
# sheetName = 'ReachStageAsDof' 
gaugeStageFileName = 'wildcat_gauges_albers_meters.xlsx'
# sheetName = 'ReachMedianStage' # HAND reach median stage as DOF
sheetName = 'ReachOutletStage' # HAND reach outlet stage as DOF

# read gauge file
gaugeStages = pd.read_excel(gaugeStageFileName, sheet_name=sheetName) # 3 gauges
print(gaugeStages)

# Need to calculate gauge stage elevation if necessary!

# keep only necessary fields from gauges
keptFields = ['stationid','x','y','stage_elevation']
gaugeWithStageElevations = gaugeStages[keptFields]
print(gaugeWithStageElevations)

### Snap Gauges to FSPs and Calculate Gauge DOF

Here we snap gauges (with their stage elevation) to FLDPLN flood source pixels (FSPs), which are the stream pixels. Each snapped gauge FSP has a stream bed elevaltion, which is used to calculate the depth of flow/flood (DOF) at those FSPs. 

This process also identifies the FLDPLN libraries that the gauges belong to. Note that the same gauges can be snapped to more than one library as FLDPLN libraries may overlap and the overalpping FSPs may have different coordinates! 

In [ ]:
# snap gauges to FSPs on-the-fly
print('Snap gauges to FSPs ...')
print(f'Number of gauges: {len(gaugeWithStageElevations.index)}')

# FLDPLN libraries to whose FSPs gauges are sanpped. All the libraries by default but can be a subset
libs2Map = ['seglib_py']

# snap the gauges to FSPs. 
# Fields 'StrOrd','DsDist','SegId','FilledElev'are used for interpolating other FSP DOF
# Note that 'lib_name','FspX', 'FspY' together uniquely identify a FSP (as there are overlapping FSPs between libraries)!
gaugeFspDf = SnapGauges2Fsps(libFolder,libs2Map,gaugeWithStageElevations,snapDist=350,gaugeXField='x',gaugeYField='y',fspColumns=['FspX','FspY','StrOrd','DsDist','SegId','FilledElev']) 
print(gaugeFspDf)

# calculate gauge FSP's DOF
gaugeFspDf['Dof'] = gaugeFspDf['stage_elevation'] - gaugeFspDf['FilledElev']

# keep only necessary columns for gauge FSPs
gaugeFspDf = gaugeFspDf[['lib_name','FspX','FspY','StrOrd','DsDist','SegId','FilledElev','Dof']] # Note that 'lib_name','FspX', 'FspY' together uniquely identify a FSP!!!

# show info
print(f'Number of snapped gauge FSPs: {len(gaugeFspDf)}')
# Find libs where the gauges are snapped to, and they are the actual libs to map
libs2Map = gaugeFspDf['lib_name'].drop_duplicates().tolist()
print(f'Libraries gauges snapped to: {libs2Map}')
print(gaugeFspDf)

#
# save snapped gauges to CSV file for checking
# gaugeFspDf.to_csv(os.path.join(outputFolder, 'SnappedGauges.csv'), index=False)

## Interpolate FSP's DOF

Here we interpolate the DOF for all the FSPs between the gauge-FSPs using their DOF calculated from previous step. The interpolation uses stream orders and starts from low stream order (i.e., main streams) to high stream order (i.e., tributatried). Either horizontal or vertical (by default) interpolation can be used.

In [ ]:
# Find libs with snapped gauges. They are the actual libs to map
libs2Map = gaugeFspDf['lib_name'].drop_duplicates().tolist()

# prepare the DF for storing interpolated FSP DOF
fspDof = pd.DataFrame(columns=['LibName','FspId','Dof'])

# prepare DFs for saving interpolated FSPs and their segment IDs
fspCols = fspInfoColumnNames + ['Dof']
segIdCols = ['SegId','LibName']
fsps = pd.DataFrame(columns=fspCols)
segIds =pd.DataFrame(columns=segIdCols)

# map each library
for libName in libs2Map:
    # interpolate DOF for the gauges
    # print('Interpolate FSP DOF using gauge DOF ...')
    # fspIdDof = InterpolateFspDofFromGauge(libFolder,libName,gaugeFspDf) # 'V' by default
    fspIdDof = InterpolateFspDofFromGauge(libFolder,libName,gaugeFspDf,weightingType='H') # horizontal interpolation
    fspIdDof['LibName'] = libName
    # fspDof = fspDof.append(fspIdDof[['LibName','FspId','Dof']], ignore_index=True)
    fspDof = pd.concat([fspDof,fspIdDof[['LibName','FspId','Dof']]], ignore_index=True)

    # Keep interpolated FSP DOF for saving later
    fspFile = os.path.join(libFolder, libName, fspInfoFileName)
    fspDf = pd.read_csv(fspFile) 
    fspDf = pd.merge(fspDf,fspDof,how='inner',on=['FspId'])
    # fsps = fsps.append(fspDf, ignore_index=True)
    fsps = pd.concat([fsps,fspDf], ignore_index=True)
    
    # Keep FSP segment IDs for saving later
    t =  pd.DataFrame(fspDf['SegId'].drop_duplicates().sort_values())
    t['LibName'] = libName
    # segIds = segIds.append(t, ignore_index=True)
    segIds = pd.concat([segIds,t], ignore_index=True)

# show interpolated FSPs with Dof
print(fspDof)

#
# save interpolated FSP DOF and their segments for checking. This block of code should be commented out if no-checking needed
#
# Save DOF and segment IDs to CSV files
FspDofFile = os.path.join(outputFolder, 'Interpolated_FSP_DOF.csv')
SegIdFile = os.path.join(outputFolder, 'Interpolated_SegIds.csv')
fsps.to_csv(FspDofFile, index=False)
segIds.to_csv(SegIdFile, index=False)

# # turn interpolated sgements into a shapefile
# for libName in libs2Map:
#     segShp = os.path.join(libFolder, libName, 'stream_orders.shp')
#     segs = gpd.read_file(segShp)
#     segs['LibName'] = libName
#     # print(segs)
#     # join by two fields: SegId and LibName
#     segDf = pd.merge(segs,segIds,how='inner',on=['SegId','LibName'])
#     # print(segDf)
#     # write segments as a shapefile
#     segDf.to_file(os.path.join(outputFolder, 'Interpolated_Segements.shp'))

## Map Flood Inundation Depth


### Set Mapping Parameters

Setup the map folder (i.e., outMapFolderName) which is under the output folder and contains all inundation depth maps. Additional settings include whether to mosaic tiles as single COG file and whether use a Dask local cluster to speed up the mapping.

In [ ]:
# set up map folder
outMapFolderName = 'wildcat_sep_03' # Wildcat Labor Day flood

# Create folders for storing temp and output map files
outMapFolder,scratchFolder = CreateFolders(outputFolder,'scratch',outMapFolderName)

# whether mosaci tiles as a single COG
mosaicTiles = True #True #False

# Using LocalCluster by default
useLocalCluster = False # This doesn't work on my office desktop though it works fine on KBS server
numOfWorkers = round(0.8*os.cpu_count())
numOfWorkers = 6
print(f'Number of workers: {numOfWorkers}')

### Map Inundation Depth

The process of generating inundation depth map happens here.

In [ ]:
# show mapping info
print(f'Tiled FLDPLN library folder: {libFolder}')
print(f'Map folder: {outMapFolder}')
# Find libs needs mapping
libs2Map = fspDof['LibName'].drop_duplicates().tolist()
print(f'Libraries to map: {libs2Map}')

# check running time
startTimeAllLibs = time.time()

# create a local cluster to speed up the mapping. Must be run inside "if __name__ == '__main__'"!!!
if useLocalCluster:
    # cluster = LocalCluster(n_workers=4,processes=False)
    try:
        print('Start a LocalCluster ...')
        # NOTE: set worker space (i.e., local_dir) to a folder that the LocalCluster can access. When run the script through a scheduled task, 
        # the system uses C:\Windows\system32 by default, which a typical user doesn't have the access!
        # cluster = LocalCluster(n_workers=numOfWorkers,memory_limit='32GB',local_dir="D:/projects_new/fldpln/tools") # for KARS production server (192G RAM & 8 cores)
        # cluster = LocalCluster(n_workers=numOfWorkers,processes=False) # for KARS production server (192G RAM & 8 cores)
        cluster = LocalCluster(n_workers=numOfWorkers,memory_limit='8GB',local_dir="E:\temp") # for office desktop (64G RAM & 8 cores)
        # print('Watch workers at: ',cluster.dashboard_link)
        print(f'Number of workers: {numOfWorkers}')
        client = Client(cluster)
        # print scheduler info
        # print(client.scheduler_info())
    except:
        print('Cannot create a LocalCLuster!')
        useLocalCluster = False

# dict to store lib processing time
libTime={}

# map each library
for libName in libs2Map:
    # check running time
    startTime = time.time()
    
    # select the FSPs within the lib
    fspIdDof = fspDof[fspDof['LibName']==libName][['FspId','Dof']]

    # mapping flood depth
    if useLocalCluster:
        print(f'Map [{libName}] using LocalCLuster ...')
        # generate a DAG
        dag,dagRoot=MapFloodDepthWithTilesAsDag(libFolder,libName,'snappy',outMapFolder,fspIdDof,aoiExtent=None)
        if dag is None:
            tileTifs = None
        else:
            # visualize DAG
            # visualize(dag)
            # Compute DAG
            tileTifs = client.get(dag, dagRoot)
            if not tileTifs: # list is empty
                tileTifs =  None
    else:
        print(f'Map {libName} ...')
        tileTifs = MapFloodDepthWithTiles(libFolder,libName,'snappy',outMapFolder,fspIdDof,aoiExtent=None)
    print(f'Actual mapped tiles: {tileTifs}')

    # Mosaic all the tiles from a library into one tif file
    if mosaicTiles and not(tileTifs is None):
        print('Mosaic tile maps ...')
        mosaicTifName = libName+'_'+outMapFolderName+'.tif'
        # Simplest implementation, may crash with very large raster
        MosaicGtifs(outMapFolder,tileTifs,mosaicTifName,keepTifs=False)
    
    # check time
    endTime = time.time()
    usedTime = round((endTime-startTime)/60,3)
    libTime[libName] = usedTime
    # print(f'{libName} processing time (minutes):', usedTime)

# Show processing time
# Individual lib processing time
print('Individual library mapping time:', libTime)
# total time
endTimeAllLibs = time.time()
print('Total processing time (minutes):', round((endTimeAllLibs-startTimeAllLibs)/60,3))

#
# Shutdown local clusters
#
if useLocalCluster:
    print('Shutdown LocalCluster ...')
    cluster.close()
    client.shutdown()
    client.close()
    useLocalCluster = False